In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
!pip install openpyxl 
!pip install bnlp_toolkit


You should consider upgrading via the 'c:\users\ziaul\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.



  Using cached gensim-4.0.1-cp37-cp37m-win_amd64.whl (23.9 MB)
  Using cached Cython-0.29.21-cp37-cp37m-win_amd64.whl (1.6 MB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.14
    Uninstalling Cython-0.29.14:
      Successfully uninstalled Cython-0.29.14
  Attempting uninstall: gensim
    Found existing installation: gensim 3.8.3
    Uninstalling gensim-3.8.3:
      Successfully uninstalled gensim-3.8.3


You should consider upgrading via the 'c:\users\ziaul\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [5]:
df = pd.read_excel('dataset/multi class bangla social media comment.xlsx')
df.to_csv('dataset/multi_class_bangla_social_media_comment.csv', encoding='utf-8', index=False)
df.head()

,comment,Category,Gender,comment react number,label
0,ওই হালার পুত এখন কি মদ খাওয়ার সময় রাতের বেলা...,Actor,Female,1.0,sexual
1,ঘরে বসে শুট করতে কেমন লেগেছে? ক্যামেরাতে কে ছি...,Singer,Male,2.0,not bully
2,"অরে বাবা, এই টা কোন পাগল????",Actor,Female,2.0,not bully
3,ক্যাপ্টেন অফ বাংলাদেশ,Sports,Male,0.0,not bully
4,পটকা মাছ,Politician,Male,0.0,troll
